# Trabalho Final - Avaliação e Desempenho 2022.1
## Grupo:
- Matheus Fernandes Cabral - DRE: 116033208
- Rafael da Silva Fernandes - DRE: 117196229
- Stephanie Orazem Hoegemann Ramos - DRE: 113278168

## Objetivo 
O foco deste trabalho de simulação será a implementação de uma simulação orientada a eventos discretos e que permita a obtenção de intervalos de confiança para algumas métricas de uma fila M/M/1, usando as disciplinas de atendimento entre FCFS e LCFS.

## Descrição dos eventos

### Chegada

- Insere o freguês na fila de fregueses que chegaram ao sistema
- Programa um evento de chegada na fila de eventos, tempo dependendo da taxa de chegada Poisson
    - Se fila vazia:
        - programa um evento de entrada no serviço da fila de eventos, com o mesmo tempo dessa chegada
    - Do contrário:
        - insere o freguês na fila de espera

### Entrada em serviço

- Programa um evento de saída do serviço, com o tempo dado pela distribuição exponencial do serviço.
- Coleta estatísticas sobre o temo de espera na fila

### Saída de serviço

- Coleta estatísticas sobre o tempo de serviço
- Se a fila não estiver vazia, programa um evento de entrada em serviço na fila de eventos, com o mesmo tempo da saída de serviço

> ToDo:
- determinar método para o término da fase transiente de cada utilização (0.2 , 0.4 , 0.6 , 0.8 , 0.9)
- mostrar independência da semente inicial
- incrementar KMIN de 100 em 100 até que sobreposição completa ocorra
    - entender como verificar se há ou não sobreposição completa
    - documentar evolução do IC da chi-quadrado a cada incremento

## Bibliotecas utilizadas

In [1]:
import numpy as np
import pandas as pd
import scipy.special as sc
from scipy.stats import chi2, t

## Variáveis globais

In [2]:
# Inicialização do identificador único dos fregueses
CUSTOMER_ID = 0

# Booleano para indicar se o servidor está ocupado (True) ou não (False)
BUSY_SERVER = False

# Número de rodadas, definido pelo enunciado do trabalho
N_ROUNDS = 3200

# Kmin >> 1, para que a média do tempo de espera por rodada possa ser assumido como distribuição Normal
KMIN = 1000

# Lista com os valores de lambda a serem trabalhos
LAMBDAS = [0.2, 0.4, 0.6, 0.8, 0.9]

# Listas
# - Lista de fregueses
CUSTOMER_LIST = []

# - Lista de eventos
EVENT_LIST = []

# - Fila de espera
WAIT_QUEUE = []

# - Estatísticas necessárias para avaliação do simulador
STATISTICS = []

## Classes

### Freguês

In [3]:
class Customer:
    '''
    Atributos do freguês:
    - id: identificador único de cada freguês
    - arrival_time: tempo de chegada no sistema
    - entry_server_time: tempo de entrada em serviço
    - exit_server_time: tempo de saída do serviço
    - arrival_round: rodada em que o freguês chega
    '''

    def __init__(self, id, arrival_time, arrival_round):
        self.id = id
        self.arrival_time = arrival_time
        self.arrival_round = arrival_round
        self.entry_server_time = 0
        self.exit_server_time = 0

### Estatísticas

In [4]:
class Statistics:
    '''
    Somatório das estatísticas, sendo elas:
    - tempo de serviço
    - tempo de espera na fila
    - tempo total gasto no sistema
    '''

    def __init__(self):
        self.sample_index = 0
        self.sample_service_time = 0
        self.sample_queue_time = 0
        self.sample_system_time = 0
        self.mean_queue_wait = 0

    
    def statistics_accumulator(self, customer):
        self.sample_index += 1
        self.sample_service_time += customer.exit_server_time - customer.entry_server_time
        self.sample_queue_time += customer.entry_server_time - customer.arrival_time
        self.sample_system_time += customer.exit_server_time - customer.arrival_time

    
    def mean_calculator(self):
        '''
        Método responsável pelo cálculo da média do tempo de espera na fila
        '''
        self.mean_queue_wait = self.sample_queue_time/ self.sample_index

### Utilitários

In [5]:
class Utils:

    def append_event(event, event_list):
        '''
        Adiciona evento à lista de eventos
        '''
        
        for i in range(len(event_list)):
            
            # Caso o evento na posição i da lista de eventos tenha um tempo de início superior ao do evento passado como parâmetro
            # Nós inserimos o evento passado como parâmetro na posição i da lista, substituindo assim o original
            if(event_list[i].start_time >= event.start_time):
                event_list.insert(i, event)
                return
        
        event_list.append(event)


    def find_customer(customer_list, id):
        '''
        Encontra freguês na lista de fregueses com o id passado como parâmetro 
        '''

        for customer in range(len(customer_list)):
            if(customer_list[customer].id == id):
                return customer

        return None


    def generate_arrival_time(lambda_rate):
        '''
        Calcula tempo de chegada com base na taxa lambda passada como parâmetro
        '''

        u0 = np.random.uniform()
        t0 = np.log(u0)/ (-lambda_rate)

        return t0


    # A taxa de serviço é igual à 1 pois foi dada na descrição do trabalho
    def generate_service_time():
        '''
        Calcula tempo de serviço com taxa igual à 1
        '''

        u0 = np.random.uniform()
        x0 = np.log(u0)/ (-1)

        return x0


    def variance_queue_wait_confidence_interval(estimated_variance):
        '''
        Cálculo do Intervalo de Confiança (IC) utilizando a distribuição chi-quadrado
        '''

        # Cálculo dos limites inferior e superior
        superior_limit = ((N_ROUNDS - 1) * estimated_variance) / chi2.ppf(q = 0.025, df = N_ROUNDS - 1)
        inferior_limit = ((N_ROUNDS - 1) * estimated_variance) / chi2.ppf(q = 0.975, df = N_ROUNDS - 1)

        # Cálculo da precisão
        chi_sup = chi2.ppf(q = 0.025, df = N_ROUNDS-1)
        chi_inf =  chi2.ppf(q = 0.975, df = N_ROUNDS-1)
        precision = (chi_sup - chi_inf)/ (chi_sup + chi_inf)

        return inferior_limit, superior_limit , precision


    def mean_queue_wait_confidence_interval(standard_deviation, estimated_mean):
        '''
        Cálculo do IC usando a distribuição t-student
        '''

        # Cálculo dos limites inferior e superior
        superior_limit = estimated_mean + (t.ppf(q = 0.975, df = N_ROUNDS - 1) * (standard_deviation/ np.sqrt(N_ROUNDS))) 
        inferior_limit = estimated_mean - (t.ppf(q = 0.975, df = N_ROUNDS - 1) * (standard_deviation/ np.sqrt(N_ROUNDS)))
        
        # Cálculo da precisão
        precision = (superior_limit - inferior_limit)/ (superior_limit + inferior_limit)
        
        return inferior_limit, superior_limit, precision

### Evento

In [6]:
class Event:
    '''
    Atributos de cada evento:
    - event_type: tipo do evento
        - chegada no sistema (CH)
        - entrada em serviço (ES)
        - saída de serviço (SS)
    - start_time: tempo de início do evento
    - customer_index: índice do freguês a qual o evento se refere
    '''

    def __init__(self, event_type, start_time, customer_index):
        self.event_type = event_type
        self.start_time = start_time
        self.customer_index = customer_index

    
    def queue_arrival(self, customer_list, event_list, wait_queue, current_round, lambda_rate):
        '''
        Fila de chegada
        '''

        # Permite alterar o valor da variável global CUSTOMER_ID dentro da função
        global CUSTOMER_ID

        arrival_time = self.start_time + Utils.generate_arrival_time(lambda_rate)

        # Incrementa índice do freguês, o instancia e adiciona na lista de fregueses
        CUSTOMER_ID += 1
        customer_list.append(Customer(CUSTOMER_ID, arrival_time, current_round))

        # Adiciona chegada no sistema à fila de eventos
        Utils.append_event(Event('CH', arrival_time, CUSTOMER_ID), event_list)

        # Caso não haja ninguém na fila de espera e o servidor não esteja vazio
        # Adiciona entrada em serviço à fila de eventos
        if(len(wait_queue) == 0 and not BUSY_SERVER):
            Utils.append_event(Event('ES', self.start_time, self.customer_index), event_list)
        
        # Caso contrário, adiciona freguês à lista de espera
        else:
            wait_queue.append(self.customer_index)


    def service_entry(self, customer_list, event_list, wait_queue, FCFS = True):
        '''
        Calcula a entrada do serviço
        '''

        global BUSY_SERVER

        # Caso a fila não esteja vazia, pega-se o freguês com base na disciplina
        if(len(wait_queue) > 0):
            
            # Com base em FCFS, pega-se o primeiro freguês
            if(FCFS):
                wait_queue.pop(0)
            # Com base em LCFS, pega-se o último freguês
            else:
                wait_queue.pop(-1)
        
        # Calcula tempo de serviço total
        service_time = self.start_time + Utils.generate_service_time()
        
        # Adiciona saída de serviço à fila de eventos
        Utils.append_event(Event('SS', service_time, self.customer_index), event_list)

        # Sinaliza que o servidor está ocupado
        BUSY_SERVER = True

        # Relaciona o freguês com o seu tempo de serviço
        customer_list[Utils.find_customer(customer_list, self.customer_index)].entry_server_time = self.start_time


    def service_exit(self, customer_list, event_list, wait_queue, statistics, current_round):
        '''
        Calcula a saída do serviço
        '''

        global BUSY_SERVER

        # Caso a fila não esteja vazia, adiciona entrada em serviço à fila de eventos
        if(len(wait_queue) > 0):
            Utils.append_event(Event('ES', self.start_time, wait_queue[0]), event_list)

        # Sinaliza que o servidor não está ocupado
        BUSY_SERVER = False

        # Calculam as estatísticas do freguês e o remove da lista de fregueses 
        aux_customer_id = Utils.find_customer(customer_list, self.customer_index)
        customer_list[aux_customer_id].exit_server_time = self.start_time
        statistics[current_round].statistics_accumulator(customer_list[aux_customer_id])
        customer_list.pop(aux_customer_id)

## Execução do simulador

### Definindo a primeira chegada no sistema

In [7]:
FIRST_CUSTOMER = Customer(0, 0, 0)
CUSTOMER_LIST.append(FIRST_CUSTOMER)

In [8]:
FIRST_ARRIVAL = Event('CH', 0, 0)
EVENT_LIST.append(FIRST_ARRIVAL)

### Fluxo principal

In [9]:
def main(lambda_rate, FCFS = True):

    # Inicialização das variáveis média e variância estimadas
    estimated_mean, estimated_variance = 0, 0

    # Lista para os resultados que devem ser retornados
    results_dict = {}

    for current_round in range( N_ROUNDS):

        STATISTICS.append(Statistics())

        while(STATISTICS[current_round].sample_index < KMIN):
            
            # Pega primeiro evento da lista de eventos
            current_event = EVENT_LIST.pop(0)

            # Caso o tipo do evento seja chegada ao sistema
            if(current_event.event_type == 'CH'):
                current_event.queue_arrival(CUSTOMER_LIST, EVENT_LIST, WAIT_QUEUE, current_round, lambda_rate)

            # Caso o tipo do evento seja entrada em serviço
            elif(current_event.event_type == 'ES'):

                # Com base em FCFS
                if(FCFS):
                    current_event.service_entry(CUSTOMER_LIST, EVENT_LIST, WAIT_QUEUE)

                # Com base em LCFS
                else:
                    current_event.service_entry(CUSTOMER_LIST, EVENT_LIST, WAIT_QUEUE, FCFS = False)

            # Caso o tipo do evento seja saída do serviço
            elif(current_event.event_type == 'SS'):
                current_event.service_exit(CUSTOMER_LIST, EVENT_LIST, WAIT_QUEUE, STATISTICS, current_round)

        # Cálcula média das estatísticas para a rodada atual
        STATISTICS[current_round].mean_calculator()


    # Cálculo da média estimada
    for statistic in STATISTICS:
        estimated_mean += statistic.mean_queue_wait

    # Cálculo da média real
    real_mean = estimated_mean/  N_ROUNDS
    results_dict["Real mean"] = real_mean

    # Cálculo da variância
    for statistic in STATISTICS:
        estimated_variance += (statistic.mean_queue_wait - real_mean) ** 2

    results_dict["Estimated variance"] = estimated_variance

    # Cálculo dos limites inferior, superior e precisão da distribuição chi-quadrado
    infe_limit, sup_limit, chi_precision = Utils.variance_queue_wait_confidence_interval(estimated_variance)
    results_dict["Chi Inferior limit"] = infe_limit
    results_dict["Chi Superior limit"] = sup_limit
    results_dict["Chi Precision"] = chi_precision

    # Cálculo dos limites inferior, superior e precisão da distribuição t-student
    mean_infe_limit, mean_sup_limit, t_precision = Utils.mean_queue_wait_confidence_interval(\
        np.sqrt(estimated_variance/ (N_ROUNDS - 1)), real_mean
    )
    results_dict["T Student Inferior limit"] = mean_infe_limit
    results_dict["T Student Superior limit"] = mean_sup_limit
    results_dict["T Student Precision"] = t_precision

    results_dataframe = pd.DataFrame.from_dict(results_dict, orient = "index")

    return results_dataframe

## Resultados gerais

### FCFS

In [10]:
%%time
results_fcfs = pd.DataFrame()

for lambda_rate in LAMBDAS:
    results_fcfs[str(lambda_rate)] = main(lambda_rate)

results_fcfs

CPU times: total: 42.3 s
Wall time: 42.4 s


,0.2,0.4,0.6,0.8,0.9
Real mean,0.250288,0.250288,0.250288,0.250288,0.250288
Estimated variance,4.240485,204.701561,405.162637,605.623713,806.084788
Chi Inferior limit,4.040111,195.028870,386.017630,577.006389,767.995148
Chi Superior limit,4.456220,215.115731,425.775243,636.434754,847.094266
Chi Precision,-0.048975,-0.048975,-0.048975,-0.048975,-0.048975
T Student Inferior limit,0.249026,0.241520,0.237953,0.235207,0.232889
T Student Superior limit,0.251550,0.259056,0.262623,0.265369,0.267687
T Student Precision,0.005042,0.035031,0.049284,0.060255,0.069515


### LCFS

In [11]:
%%time
results_lcfs = pd.DataFrame()

for lambda_rate in LAMBDAS:
    results_lcfs[str(lambda_rate)] = main(lambda_rate)

results_lcfs

CPU times: total: 109 ms
Wall time: 99 ms


,0.2,0.4,0.6,0.8,0.9
Real mean,0.250288,0.250288,0.250288,0.250288,0.250288
Estimated variance,1006.545864,1207.006940,1407.468016,1607.929091,1808.390167
Chi Inferior limit,958.983908,1149.972667,1340.961426,1531.950186,1722.938945
Chi Superior limit,1057.753777,1268.413289,1479.072800,1689.732312,1900.391823
Chi Precision,-0.048975,-0.048975,-0.048975,-0.048975,-0.048975
T Student Inferior limit,0.230846,0.228998,0.227297,0.225715,0.224228
T Student Superior limit,0.269730,0.271578,0.273279,0.274861,0.276348
T Student Precision,0.077680,0.085064,0.091856,0.098180,0.104121


## Questões

### a) Tempo médio de espera em fila

#### FCFS

#### LCFS

### b) Variância do tempo de espera em fila

#### FCFS

#### LCFS

### c) Número médio na fila de espera

#### FCFS

#### LCFS

### d) Variância do número de pessoas na fila de espera

#### FCFS

#### LCFS